In [ ]:
# isbn verifier

In [1]:
def verify(isbn):
    digs = [int(ch) if ch.isdigit() else 10
            for ch in isbn
            if ch.isdigit() or ch=='X']
    if (len(digs) != 10 or
        10 in digs[:9] or
        not (isbn[-1].isdigit() or isbn[-1]== 'X')):
        return False
    return sum([ d * (10-i) for i, d in enumerate(digs)]) % 11 == 0

In [2]:
verify('3-598-21508-9')  # False

False

In [3]:
verify('3-598-21507-X')  # True

True

In [ ]:
# pov

In [1]:
from json import dumps


class Tree(object):
    def __init__(self, label, children=[]):
        self.label = label
        self.children = children

    def __dict__(self):
        return {self.label: [c.__dict__() for c in sorted(self.children)]}

    def __str__(self, indent=None):
        return dumps(self.__dict__(), indent=indent)

    def __lt__(self, other):
        return self.label < other.label

    def __eq__(self, other):
        return self.__dict__() == other.__dict__()
    
    def pathfind(self, dict_, target, path=()):
        for k, v in dict_.items():
            path = path + (k,)
            if k == target:
                return path
            elif v:
                for subdict in v:
                    subpath = self.pathfind(subdict, target, path)
                    if subpath:
                        return subpath
        return None
    
    def pathTo(self, from_node, to_node):
        dict_ = self.__dict__()
        p_from = self.pathfind(dict_, from_node)
        p_to = self.pathfind(dict_, to_node)
        if not (p_from and p_to):
            raise ValueError(f'node {from_node if p_to else to_node} does not exist')
        while p_from and p_to and p_from[0] == p_to[0]:
            cn = p_from[0]
            p_from = p_from[1:]
            p_to = p_to[1:]
        return list(tuple(reversed(p_from)) + (cn,) + p_to)
    
    def subdict(self, node):
        dict_ = self.__dict__()
        path = list(self.pathfind(dict_, node))
        subdict = dict_[path.pop(0)]
        while path:
            nc = path.pop(0)
            for child in subdict:
                if nc in child.keys():
                    subdict = child[nc]
                    continue
        return {node: subdict}
    
    def dictFromPov(self, from_node):
        dict_ = self.__dict__()
        path = self.pathfind(dict_, from_node)
        if path:
            path = list(path)
        else:
            raise ValueError(f'node {from_node} does not exist')
        inv_pairs = list(zip(path, path[1:]))
        acc_dict = []
        while inv_pairs:
            nb, nt = inv_pairs.pop(0)
            nbc = self.subdict(nb)[nb]
            nbc = [ch for ch in nbc if nt not in ch.keys()]
            nbc.extend(acc_dict)
            acc_dict = [{nb: nbc}]
        acc_sibs = self.subdict(from_node)[from_node]
        return {from_node: acc_dict + acc_sibs}
    
    def dictToTree(self, dict_):
        k, v = list(dict_.items())[0]
        return Tree(k, [self.dictToTree(ch) for ch in v])
    
    def fromPov(self, from_node):
        return self.dictToTree(self.dictFromPov(from_node))

In [2]:
tree = Tree(0,[Tree(1,[Tree(55), 
                       Tree(4,[Tree(7)]), 
                       Tree(5,[Tree(9), Tree(11, [Tree(13)])])]),
               Tree(99)])
print(tree.__dict__())
print(tree.fromPov(4).__dict__())

{0: [{1: [{4: [{7: []}]}, {5: [{9: []}, {11: [{13: []}]}]}, {55: []}]}, {99: []}]}
{4: [{1: [{0: [{99: []}]}, {5: [{9: []}, {11: [{13: []}]}]}, {55: []}]}, {7: []}]}


In [ ]:
# dominoes

In [1]:
def chain(dominoes):
    if dominoes == []:
        return []
    chain, remainder = dominoes[:1], dominoes[1:]
    tries = [(chain, remainder)]
    while tries:
        chain, remainder = tries.pop()
        if remainder == [] and chain[0][0] == chain[-1][1]:
            return chain
        match_val = chain[-1][1]
        links = [dom for dom in remainder if match_val in dom]
        if links:
            for link in links:
                new_remainder = remainder.copy()
                new_remainder.remove(link)
                if link[0] != match_val:
                    link = tuple(reversed(link))
                tries.append((chain + [link], new_remainder))

In [2]:
input_dominoes = [(1, 2), (5, 3), (3, 1), (1, 2), (2, 4), (1, 6), (2, 3), (3, 4), (5, 6)]
output_chain = chain(input_dominoes)
print(output_chain)

[(1, 2), (2, 3), (3, 4), (4, 2), (2, 1), (1, 6), (6, 5), (5, 3), (3, 1)]


In [3]:
chain([(1, 3)])

In [4]:
chain([])

[]

In [5]:
# error handling

In [1]:
def handle_error_by_throwing_exception():
    raise Exception('somthing is wrong')


def handle_error_by_returning_none(input_data):
    try:
        return int(input_data)
    except:
        return None


def handle_error_by_returning_tuple(input_data):
    try:
        return True, int(input_data)
    except:
        return False, None


def filelike_objects_are_closed_on_exception(filelike_object):
    try:
        filelike_object.do_something()
    except:
        raise
    finally:
        filelike_object.close()


In [ ]:
# zipper

In [1]:
class Zipper(object):
    
    @staticmethod
    def from_tree(tree):
        new = Zipper()
        new.tree = tree
        new.offsets = []
        return new
    
    def value(self):
        tree = self.tree
        for offset in self.offsets:
            tree = tree[offset]
        return tree['value']

    def set_(self, key, value):
        tree = self.tree
        for offset in self.offsets:
            tree = tree[offset]
        tree[key] = value
        new = Zipper.from_tree(self.tree)
        new.offsets = self.offsets
        return new
    
    def set_value(self, value):
        return self.set_('value', value)
        
    def subtree(self):
        tree = self.tree
        for offset in self.offsets:
            tree = tree[offset]
        return tree
    
    def left(self):
        new = Zipper.from_tree(self.tree)
        new.offsets = self.offsets + ['left']
        if new.subtree():
            return new
        else:
            return None

    def set_left(self, value):
        return self.set_('left', value)

    def right(self):
        new = Zipper.from_tree(self.tree)
        new.offsets = self.offsets + ['right']
        if new.subtree():
            return new
        else:
            return None

    def set_right(self, value):
        return self.set_('right', value)

    def up(self):
        new = Zipper.from_tree(self.tree)
        if self.offsets:
            new.offsets = self.offsets[:-1]
        return new

    def to_tree(self):
        return self.tree

In [2]:
def bt(value, left, right):
    return {
        'value': value,
        'left': left,
        'right': right
        }


def leaf(value):
    return bt(value, None, None)


EMPTY_TREE = None


def create_trees():
    t1 = bt(1, bt(2, EMPTY_TREE, leaf(3)), leaf(4))
    t2 = bt(1, bt(5, EMPTY_TREE, leaf(3)), leaf(4))
    t3 = bt(1, bt(2, leaf(5), leaf(3)), leaf(4))
    t4 = bt(1, leaf(2), leaf(4))
    return (t1, t2, t3, t4)

In [3]:
t1, t2, t3, t4 = create_trees()
zipper = Zipper.from_tree(t1)      
updatedZipper = zipper.left().set_left(leaf(5))
tree = updatedZipper.to_tree()       
tree == t3

True

In [4]:
t1, t2, t3, t4 = create_trees()
zipper = Zipper.from_tree(t1)
updatedZipper = zipper.left().set_right(None)
tree = updatedZipper.to_tree()
tree == t4

True

In [ ]:
# food-chain

In [1]:
def chain():
    know = "I know an old lady who swallowed a {}."
    swallowed = "She swallowed the {} to catch the {}."
    animals_remarks = (
        ('fly', "I don't know why she swallowed the fly. Perhaps she'll die."),
        ('spider', 'It wriggled and jiggled and tickled inside her.'),
        ('bird', 'How absurd to swallow a bird!'),
        ('cat', 'Imagine that, to swallow a cat!'),
        ('dog', 'What a hog, to swallow a dog!'),
        ('goat', 'Just opened her throat and swallowed a goat!'),
        ('cow', "I don't know how she swallowed a cow!"),
        ('horse', "She's dead, of course!"),
    )

    lyrics = []
    for ari, (animal, remark) in enumerate(animals_remarks):
        lyrics.append(know.format(animal))
        lyrics.append(remark)
        if 0 < ari < 7:
            for i in range(ari, 0, -1):
                catcher = animals_remarks[i][0]
                caught = animals_remarks[i - 1][0]
                if caught == 'spider':
                    caught += ' that wriggled and jiggled and tickled inside her'
                lyrics.append(swallowed.format(catcher, caught))
            lyrics.append(animals_remarks[0][1])
        lyrics.append('')

    return '\n'.join(lyrics).strip()

In [2]:
print(chain())

I know an old lady who swallowed a fly.
I don't know why she swallowed the fly. Perhaps she'll die.

I know an old lady who swallowed a spider.
It wriggled and jiggled and tickled inside her.
She swallowed the spider to catch the fly.
I don't know why she swallowed the fly. Perhaps she'll die.

I know an old lady who swallowed a bird.
How absurd to swallow a bird!
She swallowed the bird to catch the spider that wriggled and jiggled and tickled inside her.
She swallowed the spider to catch the fly.
I don't know why she swallowed the fly. Perhaps she'll die.

I know an old lady who swallowed a cat.
Imagine that, to swallow a cat!
She swallowed the cat to catch the bird.
She swallowed the bird to catch the spider that wriggled and jiggled and tickled inside her.
She swallowed the spider to catch the fly.
I don't know why she swallowed the fly. Perhaps she'll die.

I know an old lady who swallowed a dog.
What a hog, to swallow a dog!
She swallowed the dog to catch the cat.
She swallowed th

In [ ]:
# forth

In [1]:
class StackUnderflowError(Exception):
    pass


def stack_pop(stack):
    try:
        r = stack.pop()
    except:
        raise StackUnderflowError('op needs more args')
    else:
        return r, stack

    
def do_op(op, stack):
    op = op.upper()
    x, stack = stack_pop(stack)
    if op not in ('DUP', 'DROP'):
        y, stack = stack_pop(stack)
    if op == 'DUP':
        stack.append(x)
        stack.append(x)
    elif op == 'DROP':
        pass
    elif op == 'SWAP':
        stack.append(x)
        stack.append(y)
    elif op == 'OVER':
        stack.append(y)
        stack.append(x)
        stack.append(y)
    elif op == '+':
        stack.append(y + x)
    elif op == '-':
        stack.append(y - x)
    elif op == '*':
        stack.append(y * x)
    elif op == '/':
        stack.append(y // x)
    return stack

        
def add_definition(def_string, def_dict):
    _, name, *sub, _ = def_string.split()
    if name.isdigit():
        raise ValueError('cannot redefine numbers')
    def_dict[name.upper()] = [validate(el, def_dict) for el in sub]
    return def_dict


def validate(el, def_dict):
    ops = 'DUP DROP SWAP OVER + - * /'
    if (el.isdigit() or
        el.upper() in ops or
        el.upper() in def_dict):
        return el
    else:
        raise ValueError('unknown word')
    
    
def parse(line, def_dict):
    els = [validate(el, def_dict) for el in line.split()]
    seq = []
    for el in els:
        if el.upper() in def_dict:
            seq.extend(def_dict[el.upper()])
        else:
            seq.append(el)
    return seq


def evaluate(input_data):
    stack = []
    def_dict = {}
    for line in input_data:
        if line and line[0] == ':':
            def_dict = add_definition(line, def_dict)
            line_seq = []
        else:
            line_seq = parse(line, def_dict)
        for el in line_seq:
            if el.isdigit():
                stack.append(int(el))
            else:
                stack = do_op(el, stack)
    return stack

In [2]:
input_data = [': foo dup dup ;','1 foo + +']
evaluate(input_data)

[3]

In [3]:
input_data = ['1 +']
evaluate(input_data)

StackUnderflowError: op needs more args

In [1]:
# markdown code refactoring

In [1]:
import re


def parse_markdown(markdown):
    lines = markdown.split('\n')
    res = ''
    in_list = False
    for i in lines:
        if re.match('###### (.*)', i) is not None:
            i = '<h6>' + i[7:] + '</h6>'
        elif re.match('## (.*)', i) is not None:
            i = '<h2>' + i[3:] + '</h2>'
        elif re.match('# (.*)', i) is not None:
            i = '<h1>' + i[2:] + '</h1>'
        m = re.match(r'\* (.*)', i)
        if m:
            if not in_list:
                in_list = True
                is_bold = False
                is_italic = False
                curr = m.group(1)
                m1 = re.match('(.*)__(.*)__(.*)', curr)
                if m1:
                    curr = m1.group(1) + '<strong>' + \
                        m1.group(2) + '</strong>' + m1.group(3)
                    is_bold = True
                m1 = re.match('(.*)_(.*)_(.*)', curr)
                if m1:
                    curr = m1.group(1) + '<em>' + m1.group(2) + \
                        '</em>' + m1.group(3)
                    is_italic = True
                if is_italic or is_bold:
                    i = '<ul><li>' + curr + '</li>'
                else:
                    i = '<ul><li><p>' + curr + '</p></li>'
            else:
                is_bold = False
                is_italic = False
                curr = m.group(1)
                m1 = re.match('(.*)__(.*)__(.*)', curr)
                if m1:
                    curr = m1.group(1) + '<strong>' + \
                        m1.group(2) + '</strong>' + m1.group(3)
                    is_bold = True
                m1 = re.match('(.*)_(.*)_(.*)', curr)
                if m1:
                    curr = m1.group(1) + '<em>' + m1.group(2) + \
                        '</em>' + m1.group(3)
                    is_italic = True
                if is_italic or is_bold:
                    i = '<li>' + curr + '</li>'
                else:
                    i = '<li><p>' + curr + '</p></li>'
        else:
            if in_list:
                i = '</ul>+i'
                in_list = False

        m = re.match('<h|<ul|<p|<li', i)
        if not m:
            i = '<p>' + i + '</p>'
        m = re.match('(.*)__(.*)__(.*)', i)
        if m:
            i = m.group(1) + '<strong>' + m.group(2) + '</strong>' + m.group(3)
        m = re.match('(.*)_(.*)_(.*)', i)
        if m:
            i = m.group(1) + '<em>' + m.group(2) + '</em>' + m.group(3)
        res += i
    if in_list:
        res += '</ul>'
    return res


In [2]:
import re


def md_if_headers(line):
    if re.match('###### (.*)', line) is not None:
        line = '<h6>' + line[7:] + '</h6>'
    elif re.match('## (.*)', line) is not None:
        line = '<h2>' + line[3:] + '</h2>'
    elif re.match('# (.*)', line) is not None:
        line = '<h1>' + line[2:] + '</h1>'
    return line


def md_if_markup_or_list(in_list, line):
    m = re.match(r'\* (.*)', line)
    if m:
        if not in_list:
            in_list = True
            is_bold = False
            is_italic = False

            curr = m.group(1)
            m1 = re.match('(.*)__(.*)__(.*)', curr)
            if m1:
                curr = m1.group(1) + '<strong>' + \
                    m1.group(2) + '</strong>' + m1.group(3)
                is_bold = True
            m1 = re.match('(.*)_(.*)_(.*)', curr)
            if m1:
                curr = m1.group(1) + '<em>' + m1.group(2) + \
                    '</em>' + m1.group(3)
                is_italic = True
            if is_italic or is_bold:
                line = '<ul><li>' + curr + '</li>'
            else:
                line = '<ul><li><p>' + curr + '</p></li>'
        else:
            is_bold = False
            is_italic = False
            curr = m.group(1)
            m1 = re.match('(.*)__(.*)__(.*)', curr)
            if m1:
                curr = m1.group(1) + '<strong>' + \
                    m1.group(2) + '</strong>' + m1.group(3)
                is_bold = True
            m1 = re.match('(.*)_(.*)_(.*)', curr)
            if m1:
                curr = m1.group(1) + '<em>' + m1.group(2) + \
                    '</em>' + m1.group(3)
                is_italic = True
            if is_italic or is_bold:
                line = '<li>' + curr + '</li>'
            else:
                line = '<li><p>' + curr + '</p></li>'
    else:
        if in_list:
            line = '</ul>+i'
            in_list = False
    return in_list, line


def md_close_some(res, line):
    m = re.match('<h|<ul|<p|<li', line)
    if not m:
        line = '<p>' + line + '</p>'
    m = re.match('(.*)__(.*)__(.*)', line)
    if m:
        line = m.group(1) + '<strong>' + m.group(2) + '</strong>' + m.group(3)
    m = re.match('(.*)_(.*)_(.*)', line)
    if m:
        line = m.group(1) + '<em>' + m.group(2) + '</em>' + m.group(3)
    res += line
    return res


def md_close_if_list(in_list, res):
    if in_list:
        res += '</ul>'
    return res


def parse_markdown(markdown):
    lines = markdown.split('\n')
    res = ''
    in_list = False
    for line in lines:
        line = md_if_headers(line)
        in_list, line = md_if_markup_or_list(in_list, line)
        res = md_close_some(res, line)
    res = md_close_if_list(in_list, res)  
    return res


In [3]:
parse_markdown('## Header2\n* Item 1\n* Item 2')

'<h2>Header2</h2><ul><li><p>Item 1</p></li><li><p>Item 2</p></li></ul>'

In [ ]:
# two bucket

In [1]:
def do_op(b1, b2, op):
    c1, s1 = b1
    c2, s2 = b2
    if op == 'fill':
        return (s1, s1), (c2, s2)
    elif op == 'empty':
        return (c1, s1), (0, s2)
    elif op == 'transfer':
        e2 = s2 - c2
        t = c1 if c1 <= e2 else e2
        return (c1 - t, s1), (c2 + t, s2)


def two_bucket(bucket_one_cap, bucket_two_cap, desired_liters, first):
    b1 = 0, bucket_one_cap
    b2 = 0, bucket_two_cap
    if first == 'two':
        b1, b2 = b2, b1
    op_ct = 0
    while not desired_liters in (b1[0], b2[0]):
        if b1[0] == 0:
            b1, b2 = do_op(b1, b2, 'fill')
        elif b2[0] == b2[1]:
            b1, b2 = do_op(b1, b2, 'empty')
        else:
            b1, b2 = do_op(b1, b2, 'transfer')
        op_ct += 1
    if first == 'two':
        b1, b2 = b2, b1
    if b1[0] == desired_liters:
        dl_bucket = 'one' 
        rem = b2[0]
    else:
        dl_bucket = 'two'
        rem = b1[0]
    return op_ct, dl_bucket, rem

In [2]:
two_bucket(2, 3, 3, "one")

(4, 'two', 1)

In [3]:
b1 = 0, 2
b2 = 0, 3
ops = 'fill transfer fill transfer'.split()
for oi, op in enumerate(ops):
    b1, b2 = do_op(b1, b2, op)
    print(oi, b1, b2)

0 (2, 2) (0, 3)
1 (0, 2) (2, 3)
2 (2, 2) (2, 3)
3 (1, 2) (3, 3)


In [4]:
two_bucket(7, 11, 2, 'one')

(14, 'one', 11)

In [5]:
two_bucket(7, 11, 2, 'two')

(18, 'two', 7)

In [6]:
two_bucket(3, 5, 1, "one")

(4, 'one', 5)

In [ ]:
# diffie-hellman

# learned about modular exponentiation when using a**b % c was way too slow
# found the implementation below on stack overflow

In [1]:
import random


def fermat_primality_test(number):
    for _ in range(3):
        rn = random.randint(2, number)-1
        if pow(rn, number-1, number) != 1:
            return False
    return True


def private_key(p):
    r = random.randint(2, p-1)
    while not fermat_primality_test(r):
        r = random.randint(2, p-1)
    return r


def pow_mod(x, y, z):
    "Calculate x**y % z efficiently."
    number = 1
    while y:
        if y & 1:
            number = number * x % z
        y >>= 1
        x = x * x % z
    return number


def public_key(p, g, private):
    return pow_mod(g, private, p)


def secret(p, public, private):
    return pow_mod(public, private, p)

In [2]:
m = 1_000_000
p = private_key(m)
g = private_key(m)

a = private_key(p)
A = public_key(p, g, a)
b = private_key(p)
B = public_key(p, g, b)
s = secret(p, B, a)
s2 = secret(p, A, b)

A, B, s, s==s2

(288826, 62636, 490130, True)

In [ ]:
# complex numbers

In [1]:
import math


class ComplexNumber(object):
    def __init__(self, real, imaginary):
        self.real = real
        self.imaginary = imaginary

    def __add__(self, other):
        a, b = self.real, self.imaginary
        c, d = other.real, other.imaginary
        return ComplexNumber(a+c, b+d)

    def __mul__(self, other):
        a, b = self.real, self.imaginary
        c, d = other.real, other.imaginary
        return ComplexNumber(a*c-b*d, b*c+a*d)

    def __sub__(self, other):
        a, b = self.real, self.imaginary
        c, d = other.real, other.imaginary
        return ComplexNumber(a-c, b-d)

    def __truediv__(self, other):
        a, b = self.real, self.imaginary
        c, d = other.real, other.imaginary
        real = (a*c + b*d)/(c**2 + d**2)
        imaginary = (b*c - a*d)/(c**2 + d**2)
        return ComplexNumber(real, imaginary)

    def __abs__(self):
        a, b = self.real, self.imaginary
        return math.sqrt(a**2 + b**2)

    def conjugate(self):
        a, b = self.real, self.imaginary
        return ComplexNumber(a, -b)

    def exp(self):
        exa, b = math.exp(self.real), self.imaginary
        return ComplexNumber(exa*math.cos(b), exa*math.sin(b))

    def __eq__(self, other):
        return self.real == other.real and self.imaginary == other.imaginary

    def __repr__(self):
        return f're:{self.real}, im:{self.imaginary}'

In [ ]:
# go counting

In [1]:
from collections import defaultdict


BLACK, WHITE, NONE = 'B', 'W', ''


class Board:
    """Count territories of each player in a Go game

    Args:
        board (list[str]): A two-dimensional Go board
    """

    def __init__(self, board):
        self.board = board
        self.board_dict = defaultdict(str)
        for rn, row in enumerate(board.split('\n')):
            for cn, cv in enumerate(row):
                self.board_dict[cn, rn] = cv

    def territoryFor(self, coord):
        """Find the owner and the territories given a coordinate on
           the board

        Args:
            coord ((int,int)): Coordinate on the board

        Returns:
            (str, set): A tuple, the first element being the owner
                        of that area.  One of "W", "B", "".  The
                        second being a set of coordinates, representing
                        the owner's territories.
        """
        c, r = coord
        if not self.board_dict[c, r] == ' ':
            return '', set()
        cr_offsets = ((-1, 0), (0, 1), (1, 0), (0, -1))
        unvisited = [coord]
        visited = []
        territory = set()
        bordering = set()
        while unvisited:
            c, r = unvisited.pop()
            visited.append((c, r))
            territory.add((c, r))
            for co, ro in cr_offsets:
                nc, nr = c + co, r + ro
                if (nc, nr) in visited:
                    continue
                nv = self.board_dict[nc, nr]
                if nv == ' ':
                    unvisited.append((nc, nr))
                elif nv in ('W', 'B'):
                    bordering.add(nv)
                    visited.append((nc, nr))
        stone = bordering.pop() if len(bordering) == 1 else ''
        return stone, territory

    def territories(self):
        """Find the owners and the territories of the whole board

        Args:
            none

        Returns:
            dict(str, set): A dictionary whose key being the owner
                        , i.e. "W", "B", "".  The value being a set
                        of coordinates owned by the owner.
        """
        territories = {'': set(), 'W': set(), 'B': set()}
        checked = []
        for (c, r), v in list(self.board_dict.items()):
            if v == ' ' and (c, r) not in checked:
                stone, territory = self.territoryFor((c, r))
                territories[stone].update(territory)
                checked.extend(list(territory))
        return territories


In [2]:
board5 = "\n".join([
    "  B  ",
    " B B ",
    "B W B",
    " W W ",
    "  W  "
])

board9 = "\n".join([
    "  B   B  ",
    "B   B   B",
    "WBBBWBBBW",
    "W W W W W",
    "         ",
    " W W W W ",
    "B B   B B",
    " W BBB W ",
    "   B B   "
])

In [3]:
board = Board(board5)
stone, territory = board.territoryFor((1, 4))
stone, territory

('', {(0, 3), (0, 4), (1, 4)})

In [4]:
board = Board(board5)
stone, territory = board.territoryFor((1, 1))
stone, territory

('', set())

In [5]:
board.territories()

{'': {(0, 3), (0, 4), (1, 2), (1, 4), (2, 1), (3, 2), (3, 4), (4, 3), (4, 4)},
 'B': {(0, 0), (0, 1), (1, 0), (3, 0), (4, 0), (4, 1)},
 'W': {(2, 3)}}

In [6]:
board = Board(' ')
board.territories()

{'': {(0, 0)}, 'B': set(), 'W': set()}

In [ ]:
# collatz conjecture

In [1]:
def collatz_steps(number):
    if number < 1:
        return None
    steps = 0
    while number > 1:
        if number % 2 == 0:
            number = number // 2
        else:
            number = 3 * number + 1
        steps += 1
    return steps

In [2]:
collatz_steps(12)

9

In [ ]:
# connect

In [1]:
from collections import defaultdict


class ConnectGame:
    def __init__(self, board):
        self.board = board
        self.board_dict = defaultdict(str)
        for rn, row in enumerate(board.split('\n')):
            for cn, cv in enumerate(row.split()):
                self.board_dict[rn, cn] = cv
        self.mri, self.mci = max(self.board_dict.keys())

    def get_winner(self):
        if self.mri == 0 and self.mci == 0:
            return self.board_dict[0, 0]
        
        def check():
            nonlocal winner
            vis = []
            while unvis and winner == '':
                r, c = unvis.pop()
                vis.append((r, c))
                for ro, co in move_rc_offsets:
                    tr, tc = r + ro, c + co
                    if self.board_dict[tr, tc] == win_char:
                        if (tr, tc) not in vis:
                            unvis.append((tr, tc))
                        if (tr, tc)[win_index] == win_val:
                            winner = win_char
                            break
        
        move_rc_offsets = ((-1, 0), (-1, 1), (0, 1), (1, 0), (1, -1), (0, -1))
        winner = ''
        win_char, win_index, win_val = 'O', 0, self.mri
        unvis = [(0, sc) for sc in range(self.mci+1)
                 if self.board_dict[0,sc]==win_char]
        check()
        
        win_char, win_index, win_val = 'X', 1, self.mci
        unvis = [(sr, 0) for sr in range(self.mri+1)
                 if self.board_dict[sr,0]==win_char]
        check()
        
        return winner


In [2]:
board = """. O . .
     O X X X
      O O O .
       X X O X
        . O X ."""

game = ConnectGame(board)
game.get_winner()

'O'

In [3]:
board = """ O X X X X X X X X
             O X O O O O O O O
              O X O X X X X X O
               O X O X O O O X O
                O X O X X X O X O
                 O X O O O X O X O
                   O X X X X X O X O
                    O O O O O O O X O
                     X X X X X X X X O """

game = ConnectGame(board)
game.get_winner()

'X'

In [ ]:
# two fer

In [ ]:
def two_fer(name="you"):
    return f'One for {name}, one for me.'

In [ ]:
# change

In [1]:
def find_minimum_coins(total_change, coins):
    tries = []
    for coin in coins:
        if coin == total_change:
            return [coin]
        elif coin < total_change:
            tries.append([coin])
    while tries:
        change = tries.pop(0)
        remainder = total_change - sum(change)
        for coin in coins:
            if coin == remainder:
                return sorted(change + [coin])
            elif coin < remainder:
                tries.append(change + [coin])
    return -1

In [2]:
find_minimum_coins(23, [1, 4, 15, 20, 50])

[4, 4, 15]

In [3]:
find_minimum_coins(999, [1, 2, 5, 10, 20, 50, 100])

KeyboardInterrupt: 

In [4]:
def find_minimum_coins(total_change, coins):
    if not total_change:
        return []
    elif total_change in coins:
        return [total_change]
        pass
    elif total_change < min(coins):
        return -1
    
    coins.sort()
    tries = []
    for coin in coins:
        tries.append([coin])
    ans = -1
    len_ans = total_change // min(coins) + 1
    
    while tries:
        change = tries.pop()
        if len(change) + 1 < len_ans:
            for coin in coins:
                if coin <= change[-1]:
                    if sum(change) + coin < total_change:
                        tries.append(change + [coin])
                    elif sum(change) + coin == total_change:
                        change.append(coin)
                        if len(change) < len_ans:
                            ans = sorted(change)
                            len_ans = len(change)
    return ans

In [5]:
find_minimum_coins(23, [4, 15, 20, 50])

[4, 4, 15]

In [6]:
find_minimum_coins(999, [1, 2, 5, 10, 20, 50, 100, 1000])

[2, 2, 5, 20, 20, 50, 100, 100, 100, 100, 100, 100, 100, 100, 100]

In [7]:
find_minimum_coins(3, [5, 10])

-1

In [8]:
find_minimum_coins(94, [5, 10])

-1

In [9]:
find_minimum_coins(27, [4, 5])

[4, 4, 4, 5, 5, 5]

In [ ]:
# Protein Translation

In [1]:
def proteins(strand):
    cp = ('AUG Methionine UUU,UUC Phenylalanine UUA,UUG Leucine '
          'UCU,UCC,UCA,UCG Serine UAU,UAC Tyrosine UGU,UGC Cysteine '
          'UGG Tryptophan UAA,UAG,UGA STOP').split()
    cp_dict = {}
    for c, protein in zip(cp[::2], cp[1::2]):
        for codon in c.split(','):
            cp_dict[codon] = protein
    stop = False
    proteins = []
    while strand and not stop:
        codon, strand = strand[:3], strand[3:]
        protein = cp_dict[codon]
        if protein == 'STOP':
            stop = True
        else:
            proteins.append(protein)
    return proteins


In [2]:
strand = 'UGGUGUUAUUAAUGGUUU'
expected = ['Tryptophan', 'Cysteine', 'Tyrosine']
proteins(strand) == expected

True

In [ ]:
# scale generator

In [1]:
class Scale(object):
    
    chrom_sh = 'A A# B C C# D D# E F F# G G#'.split()
    chrom_fl = 'A Bb B C Db D Eb E F Gb G Ab'.split()
    steps = {'m': 1, 'M': 2, 'A': 3}
    
    def __init__(self, tonic, scale_name, pattern=None):
        self.tonic = tonic.capitalize()
        self.scale_name = scale_name
        self.name = ' '.join([self.tonic, self.scale_name])
        if pattern:
            pattern = [self.steps[step] for step in list(pattern)]
            if sum(pattern) > 12:
                raise ValueError('pattern exceeds octave')
        self.pattern = pattern

    @property
    def pitches(self):
        if (self.tonic in 'Bb Db Eb Gb Ab D F'.split() or 
            self.scale_name == 'locrian'):
            chromatic = self.chrom_fl
        else:
            chromatic = self.chrom_sh
        tonic_index = chromatic.index(self.tonic)
        if not self.pattern:
            return chromatic[tonic_index:] + chromatic[:tonic_index]
        else:
            ind = tonic_index
            pitch_list = [chromatic[ind % 12]]
            for step in list(self.pattern)[:-1]:
                ind += step
                pitch_list.append(chromatic[ind % 12])
            return pitch_list


In [2]:
sc = Scale('c', 'chromatic')
sc.name

'C chromatic'

In [3]:
sc.pitches

['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

In [4]:
sc = Scale('G', 'major', 'MMmMMMm')
sc.name

'G major'

In [5]:
sc.pitches

['G', 'A', 'B', 'C', 'D', 'E', 'F#']

In [6]:
Scale('F', 'chromatic').pitches

['F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B', 'C', 'Db', 'D', 'Eb', 'E']

In [7]:
Scale('f#', 'minor', 'MmMMmMM').pitches

['F#', 'G#', 'A', 'B', 'C#', 'D', 'E']

In [8]:
Scale('bb', 'minor', 'MmMMmMM').pitches

['Bb', 'C', 'Db', 'Eb', 'F', 'Gb', 'Ab']

In [ ]:
# tournament

In [1]:
from collections import defaultdict


def tally(tournament_results):
    report = 'Team'.ljust(31, ' ') + '| MP |  W |  D |  L |  P\n'
    if not tournament_results:
        return report.strip()
    
    tally_dict = defaultdict(lambda : defaultdict(int))
    for game in tournament_results.split('\n'):
        p1, p2, res = game.split(';')
        tally_dict[p1]['MP'] += 1
        tally_dict[p2]['MP'] += 1
        if res == 'loss':
            p1, p2 = p2, p1
            res = 'win'
        if res == 'win':
            tally_dict[p1]['W'] += 1
            tally_dict[p1]['P'] += 3
            tally_dict[p2]['L'] += 1
        elif res == 'draw':
            for p in p1, p2:
                tally_dict[p]['D'] += 1
                tally_dict[p]['P'] += 1
    
    table = []
    for k, v in tally_dict.items():
        table.append([k.ljust(30, ' ')] + [v[vk] for vk in 'MP W D L P'.split()])
    table.sort(key=lambda r: (-r[5], r[1], r[0]))
    
    for row in table:
        report += ' |  '.join([str(val) for val in row]) + '\n'
    return report.strip()


In [2]:
results = ('Allegoric Alaskans;Blithering Badgers;win\n'
           'Devastating Donkeys;Courageous Californians;draw\n'
           'Devastating Donkeys;Allegoric Alaskans;win\n'
           'Courageous Californians;Blithering Badgers;loss\n'
           'Blithering Badgers;Devastating Donkeys;loss\n'
           'Allegoric Alaskans;Courageous Californians;win')

print(tally(results))

Team                           | MP |  W |  D |  L |  P
Devastating Donkeys            |  3 |  2 |  1 |  0 |  7
Allegoric Alaskans             |  3 |  2 |  0 |  1 |  6
Blithering Badgers             |  3 |  1 |  0 |  2 |  3
Courageous Californians        |  3 |  0 |  1 |  2 |  1


In [3]:
results = 'Allegoric Alaskans;Blithering Badgers;draw'
print(tally(results))

Team                           | MP |  W |  D |  L |  P
Allegoric Alaskans             |  1 |  0 |  1 |  0 |  1
Blithering Badgers             |  1 |  0 |  1 |  0 |  1


In [4]:
results = ''
print(tally(results))

Team                           | MP |  W |  D |  L |  P


In [ ]:
# alphametics

In [1]:
from itertools import permutations


def solve(puzzle):
    words = [w for w in puzzle.split() if w.isalpha()]
    nonzero = set([w[0] for w in words])
    letters = list(set(''.join(words))-nonzero) + list(nonzero)
    
    perms = permutations('0123456789', len(letters))
    for perm in perms:
        conv_dict = dict(zip(letters, perm))
        if '0' in perm[-len(nonzero):]:
            continue
        if eval(''.join([conv_dict[c] if c.isalpha() else c for c in puzzle])):
            return {k: int(v) for k, v in conv_dict.items()}
    
    return {}

In [2]:
%%time
puzzle = "SEND + MORE == MONEY"
soln = solve(puzzle)

CPU times: user 458 ms, sys: 4.68 ms, total: 462 ms
Wall time: 462 ms


In [3]:
soln

{'D': 7, 'E': 5, 'M': 1, 'N': 6, 'O': 0, 'R': 8, 'S': 9, 'Y': 2}

In [4]:
%%time
puzzle = "AND + A + STRONG + OFFENSE + AS + A + GOOD == DEFENSE"
soln = solve(puzzle)

CPU times: user 41 s, sys: 58.7 ms, total: 41.1 s
Wall time: 41.2 s


In [5]:
soln

{'A': 5,
 'D': 3,
 'E': 4,
 'F': 7,
 'G': 8,
 'N': 0,
 'O': 2,
 'R': 1,
 'S': 6,
 'T': 9}

In [ ]:
# word search

In [1]:
from collections import defaultdict


class Point(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Point({}:{})'.format(self.x, self.y)


class WordSearch(object):
    def __init__(self, puzzle):
        self.puz_dict = defaultdict(str)
        for ln, line in enumerate(puzzle.split('\n')):
            for cn, ch in enumerate(line):
                self.puz_dict[ln, cn] = ch
        self.search_locs = list(self.puz_dict)
    
    def search(self, word):
        dir_offsets = ((1,0), (1,1), (0,1), (-1,1), (-1,0), (-1,-1), (0,-1), (1,-1))
        match = False
        for ln, cn in self.search_locs:
            if self.puz_dict[ln, cn] == word[0]:
                ws = Point(cn, ln)
                for do in dir_offsets:
                    line_offset, col_offset = do
                    match = True
                    for wchar_num, wchar in enumerate(word):
                        check_line = ln + wchar_num * line_offset
                        check_col = cn + wchar_num * col_offset
                        if self.puz_dict[check_line, check_col] != wchar:
                            match = False
                    if match:
                        wf = Point(check_col, check_line)
                        break
            if match:
                break
        return (ws, wf) if match else None

In [2]:
puzzle = ('jefblpepre\n'
          'camdcimgtc\n'
          'oivokprjsm\n'
          'pbwasqroua\n'
          'rixilelhrs\n'
          'wolcqlirpc\n'
          'fortranftw\n'
          'alxhpburyi\n'
          'clojurermt\n'
          'jalaycalmp\n')

ws = WordSearch(puzzle)
ws.search('java')

(Point(0:0), Point(3:3))

In [3]:
ws.search('fortran')

(Point(0:6), Point(6:6))

In [4]:
ws.search('clojures')

In [ ]:
# grep

In [1]:
import os
ILIADFILENAME = 'iliad.txt'
ILIADCONTENTS = '''Achilles sing, O Goddess! Peleus' son;
His wrath pernicious, who ten thousand woes
Caused to Achaia's host, sent many a soul
Illustrious into Ades premature,
And Heroes gave (so stood the will of Jove)
To dogs and to all ravening fowls a prey,
When fierce dispute had separated once
The noble Chief Achilles from the son
Of Atreus, Agamemnon, King of men.
'''

MIDSUMMERNIGHTFILENAME = 'midsummer-night.txt'
MIDSUMMERNIGHTCONTENTS = '''I do entreat your grace to pardon me.
I know not by what power I am made bold,
Nor how it may concern my modesty,
In such a presence here to plead my thoughts;
But I beseech your grace that I may know
The worst that may befall me in this case,
If I refuse to wed Demetrius.
'''

PARADISELOSTFILENAME = 'paradise-lost.txt'
PARADISELOSTCONTENTS = '''Of Mans First Disobedience, and the Fruit
Of that Forbidden Tree, whose mortal tast
Brought Death into the World, and all our woe,
With loss of Eden, till one greater Man
Restore us, and regain the blissful Seat,
Sing Heav'nly Muse, that on the secret top
Of Oreb, or of Sinai, didst inspire
That Shepherd, who first taught the chosen Seed
'''


def create_file(name, contents):
    with open(name, 'w') as f:
        f.write(contents)

        
def remove_file(file_name):
    try:
        os.remove(file_name)
    except OSError:
        pass

    
create_file(ILIADFILENAME, ILIADCONTENTS)
create_file(MIDSUMMERNIGHTFILENAME, MIDSUMMERNIGHTCONTENTS)
create_file(PARADISELOSTFILENAME, PARADISELOSTCONTENTS)

In [2]:
def grep(pattern, files, flags=''):
    
    print_line_numbers = '-n' in flags
    print_matching_filename_only = '-l' in flags
    case_insensitive = '-i' in flags
    invert_match = '-v' in flags
    match_entire_line = '-x' in flags
    
    greport = ''
    if case_insensitive:
        pattern = pattern.lower()
    for file in files:
        with open(file) as f:
            line_ct = 0
            for line in f:
                line_ct += 1
                target = line.strip()
                if case_insensitive:
                    target = target.lower()
                match = pattern in target
                if match_entire_line:
                    match = pattern == target
                if invert_match:
                    match = not match
                if match:
                    if print_matching_filename_only:
                        greport += file + '\n'
                        break
                    else:
                        if len(files) > 1:
                            greport += f'{file}:'
                        if print_line_numbers:
                            greport += f'{line_ct}:'
                        greport += line.strip() + '\n'
    return greport

In [3]:
grep("But I beseech your grace that I may know",
     ["iliad.txt", "midsummer-night.txt", "paradise-lost.txt"],
     "-x"),

('midsummer-night.txt:But I beseech your grace that I may know\n',)

In [4]:
grep("FORBIDDEN", [PARADISELOSTFILENAME], "-i"),

('Of that Forbidden Tree, whose mortal tast\n',)

In [5]:
remove_file(ILIADFILENAME)
remove_file(MIDSUMMERNIGHTFILENAME)
remove_file(PARADISELOSTFILENAME)

In [ ]:
# clock

In [1]:
class Clock(object):
    def __init__(self, hour, minute):
        self.minutes = ((hour % 24) * 60 + minute) % (24 * 60)
        self.resolve()
        
    def resolve(self):
        self.minute = self.minutes % 60
        self.hour = (self.minutes - self.minute) // 60

    def __add__(self, other):
        if type(other) == Clock:
            self.minutes += other.minutes
        elif type(other) == int:
            self.minutes += other
        self.minutes = self.minutes % (24 * 60)
        self.resolve()
        return self
        
    def __repr__(self):
        return f'{self.hour:0>2}:{self.minute:0>2}'
    
    def __eq__(self, other):
        return self.minutes == other.minutes


In [2]:
tc = Clock(12, 30) 
tc + Clock(0, 30)

13:00

In [3]:
Clock(18,7) == Clock(-54, -11513)

True

In [4]:
Clock(18,7) == Clock(-54, -11512)

False

In [5]:
Clock(10, 0) + 3

10:03

In [6]:
Clock(12, 12)

12:12

In [ ]:
# all your base

In [1]:
def rebase(from_base, digits, to_base):
    if min(from_base, to_base) < 2:
        raise ValueError('bases must be > 1')
    if not digits or max(digits)==0:
        return []
    
    dec = 0
    for di, d in enumerate(reversed(digits)):
        if d >= from_base or d < 0:
            raise ValueError('digits must be non-negative and less than base')
        dec += from_base**di * d
    
    pwr = 0
    while to_base**pwr < dec:
        pwr += 1
    if pwr > 0:
        pwr -= 1
    
    to_base_digits = []
    while pwr >= 0:
        place_val = to_base**pwr
        pwr -= 1
        rem = dec % place_val
        to_base_digits.append((dec-rem)//place_val)
        dec = rem
    return to_base_digits

In [2]:
assert rebase(97, [3, 46, 60], 73) == [6, 10, 45]

In [3]:
rebase(2, [1], 10)

[1]

In [ ]:
# variable length quantity

In [1]:
def encode(numbers):
    result = []
    for number in numbers:
        result.extend(_encode(number))
    return result

def _encode(number):
    as_bin = bin(number)[2:]
    parts = []
    while as_bin:
        end = as_bin[-7:]
        as_bin = as_bin[:-7]
        cbit = '1' if parts else '0'
        as_hex = cbit + end.rjust(7, '0')
        parts = [as_hex] + parts
    return list(map(lambda b: int(b, 2), parts))


def decode(bytes_):
    numbers = []
    bin_strs = [bin(byt)[2:].rjust(8, '0') for byt in bytes_]
    accum = ''
    for bs in bin_strs:
        if bs[0] == '1':
            accum += bs[1:]
        else:
            accum += bs[1:]
            numbers.append(int(accum, 2))
            accum = ''
    if accum != '':
        raise ValueError('incomplete sequence')
    return numbers


decode(encode([10, 100, 1000]))

[10, 100, 1000]

In [2]:
decode([0xff])

ValueError: incomplete sequence

In [ ]:
# diamond

In [1]:
from string import ascii_uppercase

def make_diamond(letter):
    diamond = []
    letters = ascii_uppercase[:ascii_uppercase.index(letter)+1]
    start_spaces = len(letters)-1
    for sc, ch in enumerate(letters):
        ln = (' '*(start_spaces-sc) + 
              ch + 
              ' '*(2*sc-1) +
              (ch if ch != 'A' else '') +
              ' '*(start_spaces-sc)
             )
        diamond.append(ln)
    for sc, ch in enumerate(letters[-2::-1]):
        ln = (' '*(sc+1) + 
              ch + 
              ' '*(2*start_spaces-2*sc-3) +
              (ch if ch != 'A' else '') +
              ' '*(sc+1)
             )
        diamond.append(ln)
    return '\n'.join(diamond) + '\n'

print(make_diamond('E'))

    A    
   B B   
  C   C  
 D     D 
E       E
 D     D 
  C   C  
   B B   
    A    



In [2]:
from string import ascii_uppercase


def make_diamond(letter):
    letters = ascii_uppercase[:ascii_uppercase.index(letter)+1]
    size = 2*len(letters)-1
    blank = [list(' ' * size) for _ in range(size)]
    for ci, ch in enumerate(letters):
        for ri in (ci, size-1-ci):
            row = blank[ri]
            row[len(letters)-1-ci] = ch
            row[-len(letters)+ci] = ch
    return '\n'.join(list(map(lambda r: ''.join(r), blank))) + '\n'


print(make_diamond('E'))

    A    
   B B   
  C   C  
 D     D 
E       E
 D     D 
  C   C  
   B B   
    A    



In [ ]:
# reverse-string

In [1]:
def reverse(input=''):
    return ''.join(c for c in input[::-1])

In [2]:
def reverse(input=''):
    return input[::-1]

In [3]:
reverse('robot')

'tobor'

In [4]:
reverse('I\'m hungry!')

"!yrgnuh m'I"

In [5]:
reverse('racecar')

'racecar'

In [1]:
# linked list

In [1]:
class Node(object):
    def __init__(self, value, next_=None, previous=None):
        self.value = value
        self.next_ = next_
        self.previous = previous
        

class LinkedList(object):
    def __init__(self):
        self.head = None
        self.tail = None
    
    def push(self, value):
        new_head = Node(value)
        if self.head:
            new_head.next_ = self.head
            self.head.previous = new_head
        self.head = new_head
        if not self.tail:
            self.tail = self.head
            
    def unshift(self, value):
        new_tail = Node(value)
        if self.tail:
            new_tail.previous = self.tail
            self.tail.next_ = new_tail
        self.tail = new_tail
        if not self.head:
            self.head = self.tail
            
    def pop(self):
        new_head = self.head.next_
        value = self.head.value
        self.head = new_head
        if self.head:
            self.head.previous = None
        return value
    
    def shift(self):
        new_tail = self.tail.previous
        value = self.tail.value
        self.tail = new_tail
        if new_tail:
            new_tail.next_ = None
        return value


In [2]:
test = LinkedList()
test.push(10)
test.push(20)
print(test.pop())
print(test.pop())

20
10


In [3]:
test = LinkedList()
test.push(10)
test.push(20)
print(test.shift())
print(test.shift())

10
20


In [4]:
test = LinkedList()
test.unshift(10)
test.unshift(20)
print(test.shift())
print(test.shift())

20
10


In [ ]:
# simple linked list

In [1]:
class Node(object):
    def __init__(self, value):
        self._value = value
        self._next = None

    def value(self):
        return self._value

    def next(self):
        return self._next


class LinkedList(object):
    def __init__(self, values=[]):
        self._head = None
        for value in values:
            self.push(value)
            
    def __len__(self):
        length = 0
        node = self._head
        while node:
            length += 1
            node = node._next
        return length
    
    def head(self):
        if self._head:
            return self._head
        else:
            raise EmptyListException
    
    def push(self, value):
        new_head = Node(value)
        if self._head:
            new_head._next = self._head
        self._head = new_head
    
    def pop(self):
        if self._head:
            value = self._head.value()
            self._head = self._head._next
            return value
        else:
            raise EmptyListException

    def __iter__(self):
        self.iter_next = self._head
        return self
    
    def __next__(self):
        if self.iter_next:
            value = self.iter_next.value()
            self.iter_next = self.iter_next._next
            return value
        else:
            raise StopIteration
            
    def reversed(self):
        values = list(self)
        return LinkedList(values)

    
class EmptyListException(Exception):
    pass


test = LinkedList([1, 2, 3, 4])
test.push(5)
print(list(test))
print(list(test.reversed()))
print(f'test len = {len(test)}')
for _ in range(5):
    print(test.pop())
print(f'test len = {len(test)}')

[5, 4, 3, 2, 1]
[1, 2, 3, 4, 5]
test len = 5
5
4
3
2
1
test len = 0


In [2]:
sut = LinkedList([3, 4, 5])
assert sut.pop() == 5
assert len(sut) == 2
assert sut.head().value() == 4

In [3]:
sut = LinkedList([1])
assert sut.head().next() is None

In [ ]:
# book store

single copy is $8
discounts for *different* books:
- 2 - 5%
- 3 - 10%
- 4 - 20%
- 5 - 25%

ex: 1 1 2 3
- (1 2 3)@10% disc, 
- 1@$8

In [1]:
def take_unique(books):
    uniques = []
    remainder = []
    for book in books:
        if book in uniques:
            remainder.append(book)
        else:
            uniques.append(book)
    return uniques, remainder

def calculate_total(books):
    if not books:
        return 0
    
    set_prices = dict(zip(range(1, 6),
                          [8, 16*0.95, 24*0.9, 32*0.8, 40*0.75]))
    set_sizes = []
    while books:
        uniq, books = take_unique(books)
        set_sizes.append(len(uniq))
    new_price = sum([set_prices[ss] for ss in set_sizes])
    
    while max(set_sizes) - min(set_sizes) > 1:
        orig_price = new_price
        set_sizes.sort()
        set_sizes[0] += 1
        set_sizes[-1] -= 1
        new_price = sum([set_prices[ss] for ss in set_sizes])
        if orig_price < new_price:
            return orig_price
    
    return new_price

In [ ]:
# list_ops

In [1]:
def append(xs, ys):
    total_len = length(xs) + length(ys)
    output = [None] * total_len
    for xi, x in enumerate(xs):
        output[xi] = x
    for yi, y in enumerate(ys):
        output[len(xs) + yi] = y
    return output


def concat(lists):
    output = []
    for list in lists:
        output = append(output, list)
    return output


def filter_clone(function, xs):
    output = []
    for x in xs:
        if function(x):
            output = append(output, [x])
    return output


def length(xs):
    item_ct = 0
    for item in xs:
        item_ct += 1
    return item_ct


def map_clone(function, xs):
    return [function(x) for x in xs]


def foldl(function, xs, acc):
    if xs:
        arg_zero = xs[0]
        for x in xs[1:]:
            arg_zero = function(arg_zero, x)
        return function(arg_zero, acc)
    else:
        return function(1, acc)


def foldr(function, xs, acc):
    if xs:
        arg_zero = function(xs[-1], acc)
        for x in reverse(xs[:-1]):
            arg_zero = function(x, arg_zero)
        return arg_zero
    else:
        return function(acc, 1)


def reverse(xs):
    rev = xs[:]
    for xi, x in enumerate(xs):
        rev[-xi-1] = x
    return rev

assert append([1, 2], [3, 4]) == [1, 2, 3, 4]
assert append([], []) == []
assert concat([[1, 2], [3], [], [4, 5, 6]]) == [1, 2, 3, 4, 5, 6]
assert reverse([1, 2, 3, 4]) == [4, 3, 2, 1]
import operator
assert foldl(operator.add, [1, 2, 3, 4], 5) == 15
assert foldl(operator.mul, [], 2) == 2
assert foldl(operator.floordiv, [2, 5], 5) == 0
assert foldr(operator.floordiv, [2, 5], 5) == 2

In [1]:
# binary_search

In [1]:
import time

def binary_search(list_of_numbers, number):
    start_index = 0
    end_index = len(list_of_numbers) - 1
    
    while end_index >= start_index:
        middle_index = (start_index + end_index)//2
        middle_number = list_of_numbers[middle_index]
        if middle_number == number:
            return middle_index
        elif middle_number > number:
            end_index = middle_index - 1
        elif middle_number < number:
            start_index = middle_index + 1
    
    raise ValueError
       

test_list = list(range(10))
binary_search(test_list, 3)

3

In [ ]:
# poker

In [1]:
from itertools import product
from random import sample

ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
suits = 'S H C D'.split()
deck = [r+s for r, s in product(ranks, suits)]

In [14]:
from collections import Counter
from operator import itemgetter


def rank_and_suit_analysis(hand):
    ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
    rank_val_dict = dict(zip(ranks, range(2, 15)))
    suit_counts = list(Counter([card[-1] for card in hand]).values())
    ranks_counter = Counter([rank_val_dict[card[:-1]] for card in hand])
    ranks_mc = sorted(ranks_counter.most_common(),
                      key=itemgetter(1, 0), reverse=True)
    ranks_for_counts, rank_counts = zip(*ranks_mc)
    return suit_counts, rank_counts, ranks_for_counts


def is_sequential(ranklist):
    return sorted(ranklist) == list(range(min(ranklist), max(ranklist)+1))


def score(hand):
    suit_counts, rank_counts, ranks_for_counts = rank_and_suit_analysis(hand)
    if suit_counts == [5] and is_sequential(ranks_for_counts):
        hand_score = 9  # straight flush
    elif rank_counts == (4, 1):
        hand_score = 8  # four of a kind
    elif rank_counts == (3, 2):
        hand_score = 7  # full house
    elif suit_counts == [5]:
        hand_score = 6  # flush
    elif is_sequential(ranks_for_counts):
        hand_score = 5  # straight
    elif rank_counts == (3, 1, 1):
        hand_score = 4  # three of a kind
    elif rank_counts == (2, 2, 1):
        hand_score = 3  # two pair
    elif rank_counts == (2, 1, 1, 1):
        hand_score = 2  # pair
    else:
        hand_score = 1  # high card
    return hand_score, ranks_for_counts


def poker(hands):
    scored_hands = sorted([(score(hand), hand) for hand in hands])
    max_score = scored_hands[-1][0]
    return [hand
            for score, hand in scored_hands
            if score == max_score]


In [3]:
a = '4S 5H 5S 5D 5C'.split()
b = '7S 8S 9S 6S 5S'.split()
print(score(a))
print(score(b))
poker([a,b])

(8, (5, 4))
(5, (9, 8, 7, 6, 5))


[['4S', '5H', '5S', '5D', '5C']]

In [4]:
fourOf3low = '3S 3H 2S 3D 3C'.split()
fourOf3high = '3S 3H 4S 3D 3C'.split()

poker([fourOf3high, fourOf3low])

[['3S', '3H', '4S', '3D', '3C']]

In [5]:
sflush = '8S 9S 10S JS QS'.split()
foak = '4S 4C 4D 4H 10H'.split()
fullh = '3S 3D 5D 5H 5C'.split()
flush = '2C 4C 7C JC AC'.split()
straight = '3S 4H 5D 6H 7C'.split()
throak = '2S 4C 4D 4H 10H'.split()
twop = '8S 8D 5D 5H JC'.split()
pair = '3S 4S 4D 6H 7C'.split()
high = '2S 4S 5D 8H JH'.split()

In [6]:
rand = sample(deck, 5)
rand2 = sample(deck, 5)
poker([rand, rand2, twop, pair])

[['8S', '8D', '5D', '5H', 'JC']]

9: straight flush: sequential ranks, same suit
8: four of a kind: four of same rank, kicker wins tie
7: full house: three of a kind, and one pair
6: flush: five cards of suit, non-sequential
5: straight: five sequential cards of differing suits
4: three of a kind: kickers break tie
3: two pair: kbt
2: one pair: 
1: high card:

In [ ]:
# tree-building

In [1]:
class Record():
    def __init__(self, record_id, parent_id):
        self.record_id = record_id
        self.parent_id = parent_id


class Node():
    def __init__(self, node_id):
        self.node_id = node_id
        self.children = []


def BuildTree(records):
    root = None
    records.sort(key=lambda x: x.record_id)
    ordered_id = [i.record_id for i in records]
    if records:
        if ordered_id[-1] != len(ordered_id) - 1:
            raise ValueError
        if ordered_id[0] != 0:
            raise ValueError
    
    trees = []
    parent = {}
    for i in range(len(ordered_id)):
        for j in records:
            if ordered_id[i] == j.record_id:
                if j.record_id == 0:
                    if j.parent_id != 0:
                        raise ValueError
                if j.record_id < j.parent_id:
                    raise ValueError
                if j.record_id == j.parent_id:
                    if j.record_id != 0:
                        raise ValueError
                trees.append(Node(ordered_id[i]))
    for i in range(len(ordered_id)):
        for j in trees:
            if i == j.node_id:
                parent = j
        for j in records:
            if j.parent_id == i:
                for k in trees:
                    if k.node_id == 0:
                        continue
                    if j.record_id == k.node_id:
                        child = k
                        parent.children.append(child)
    if len(trees) > 0:
        root = trees[0]
    return root


In [2]:
def find_parent(root, p_id):
    to_visit = [root]
    while to_visit:
        node = to_visit.pop()
        if node.node_id == p_id:
            return node
        else:
            to_visit.extend(node.children)
    raise ValueError

    
def BuildTree(records):
    if not records:
        return None
    if not sorted([r.record_id for r in records]) == list(range(len(records))):
        raise ValueError
    
    records.sort(key=lambda x: (x.parent_id, x.record_id))
    root = Node(0)
    for rec in records[1:]:
        if rec.parent_id >= rec.record_id:
            raise ValueError
        node = find_parent(root, rec.parent_id)
        node.children.append(Node(rec.record_id))
    return root

In [3]:
records = [
            Record(6, 2),
            Record(0, 0),
            Record(3, 1),
            Record(2, 0),
            Record(4, 1),
            Record(5, 2),
            Record(1, 0)
        ]

root = BuildTree(records)

In [ ]:
# transpose

In [1]:
from itertools import zip_longest


def transpose(input_lines):
    if not input_lines:
        return ''
    inp = [line.replace(' ', '~') for line in input_lines.split('\n')]
    transpose = list(map(lambda z: ''.join(z),
                         zip_longest(*inp, fillvalue=' ')))
    transpose = [line.rstrip().replace('~', ' ') for line in transpose]
    return '\n'.join(transpose)

In [2]:
print(transpose('this\nis\nnot so hard\nis it'))

tini
hsos
i t 
s  i
  st
  o
   
  h
  a
  r
  d


In [ ]:
# dot_ds

In [1]:
NODE, EDGE, ATTR = range(3)


class Node(object):
    def __init__(self, name, attrs={}):
        self.name = name
        self.attrs = attrs

    def __eq__(self, other):
        return self.name == other.name and self.attrs == other.attrs


class Edge(object):
    def __init__(self, src, dst, attrs={}):
        self.src = src
        self.dst = dst
        self.attrs = attrs

    def __eq__(self, other):
        return (self.src == other.src and
                self.dst == other.dst and
                self.attrs == other.attrs)


def validate_graph_data(data):
    if not type(data) == list or min(map(len, data)) < 3:
        raise TypeError
    for k, *d in data:
        dtypes = list(map(type, d))
        if k not in (ATTR, NODE, EDGE):
            raise ValueError
        elif (
                (k == ATTR and dtypes != [str, str]) or
                (k == NODE and not (dtypes == [str, dict] or
                                    dtypes == [str, set])) or
                (k == EDGE and dtypes != [str, str, dict])
             ):
            raise ValueError


class Graph(object):
    def __init__(self, data=[]):
        self.attrs = {}
        self.nodes = []
        self.edges = []
        if data:
            validate_graph_data(data)
            self.assign(data)

    def assign(self, data):
        for k, *d in data:
            if k == ATTR:
                self.attrs[d[0]] = d[1]
            elif k == NODE:
                self.nodes.append(Node(*d))
            elif k == EDGE:
                self.edges.append(Edge(*d))


In [2]:
data = [(ATTR, "foo", "1"),
        (ATTR, "title", "Testing Attrs"),
        (NODE, "a", {"color": "green"}),
        (NODE, "c", {}),
        (NODE, "b", {"label", "Beta!"}),
        (EDGE, "b", "c", {}),
        (EDGE, "a", "b", {"color": "blue"}),
        (ATTR, "bar", "true"),
        ]

min(map(len, data))

3

In [3]:
g = Graph(data)

In [4]:
g.nodes

In [1]:
# triangle

In [1]:
class TriangleError(Exception):
    pass


class Triangle(object):
    def __init__(self, side_a, side_b, side_c):
        self.validate(side_a, side_b, side_c)
        self._a = side_a
        self._b = side_b
        self._c = side_c
    
    @staticmethod
    def validate(a, b, c):
        if min(a, b, c) <= 0 or max(a, b, c) >= sum([a, b, c]) - max(a, b, c):
            raise TriangleError

    def kind(self):
        uniq_to_kind = {1: 'equilateral', 2: 'isosceles', 3: 'scalene'}
        return uniq_to_kind[len({self._a, self._b, self._c})]
 

In [2]:
t = Triangle(3, 4, 5)
t.kind()

'scalene'

In [15]:
# house

In [1]:
def verse(vn):
    lines = (('the house that Jack built.', ''),
             ('the malt', 'lay in'),
             ('the rat', 'ate'),
             ('the cat', 'killed'),
             ('the dog', 'worried'),
             ('the cow with the crumpled horn', 'tossed'),
             ('the maiden all forlorn', 'milked'),
             ('the man all tattered and torn', 'kissed'),
             ('the priest all shaven and shorn', 'married'),
             ('the rooster that crowed in the morn', 'woke'),
             ('the farmer sowing his corn', 'kept'),
             ('the horse and the hound and the horn', 'belonged to'))
    verse = 'This is '
    for n in range(vn, -1 , -1):
        noun, verb = lines[n]
        verse += noun
        verse += f'\nthat {verb} ' if verb else '\n'
    return verse.strip()


def rhyme():
    return '\n\n'.join([verse(n) for n in range(12)])

In [2]:
print(verse(2))

This is the rat
that ate the malt
that lay in the house that Jack built.


In [3]:
print(rhyme())

This is the house that Jack built.

This is the malt
that lay in the house that Jack built.

This is the rat
that ate the malt
that lay in the house that Jack built.

This is the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the dog
that worried the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the cow with the crumpled horn
that tossed the dog
that worried the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the maiden all forlorn
that milked the cow with the crumpled horn
that tossed the dog
that worried the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the man all tattered and torn
that kissed the maiden all forlorn
that milked the cow with the crumpled horn
that tossed the dog
that worried the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the priest all shaven and shor

In [ ]:
# ocr_numbers

In [1]:
num_grid = ["    _  _ ",
            "  | _| _|",
            "  ||_  _|",
            "         ",
            "    _  _ ",
            "|_||_ |_ ",
            "  | _||_|",
            "         ",
            " _  _  _ ",
            "  ||_||_|",
            "  ||_| _|",
            "         ",
            ]

In [2]:
num_grid = [
            "       _     _        _  _ ",
            "  |  || |  || |  |  || || |",
            "  |  ||_|  ||_|  |  ||_||_|",
            "                           "
        ]

In [3]:
def convert(input_grid):
    line_lens_vary = len(set(map(len, input_grid))) != 1
    if line_lens_vary or len(input_grid) % 4 != 0 or len(input_grid[0]) % 3 != 0:
        raise ValueError
    
    segs_to_digit = {(' _ ', '| |', '|_|', '   '): '0',
                     ('   ', '  |', '  |', '   '): '1',
                     (' _ ', ' _|', '|_ ', '   '): '2',
                     (' _ ', ' _|', ' _|', '   '): '3',
                     ('   ', '|_|', '  |', '   '): '4',
                     (' _ ', '|_ ', ' _|', '   '): '5',
                     (' _ ', '|_ ', '|_|', '   '): '6',
                     (' _ ', '  |', '  |', '   '): '7',
                     (' _ ', '|_|', '|_|', '   '): '8',
                     (' _ ', '|_|', ' _|', '   '): '9'}
    
    row_ct = len(input_grid)//4
    out = []
    
    for drn in range(row_ct):
        dig_row = input_grid[drn*4:drn*4+4]
        row_digits_ct = len(dig_row[0])//3
        row = ''
        for rdn in range(row_digits_ct):
            digit = [line[rdn*3:rdn*3+3] for line in dig_row]
            row += segs_to_digit.get(tuple(digit), '?')
        out.append(row)
    
    return ','.join(out)

In [4]:
convert(num_grid)

'110101100'

In [ ]:
# phone_number

In [1]:
class Phone(object):
    def __init__(self, phone_number):
        self._raw = phone_number
        ph = ''.join(ch for ch in phone_number if ch.isdigit())
        ph = ph[1:] if len(ph)==11 and ph[0] == '1' else ph
        if len(ph) != 10 or int(ph[0])<2 or int(ph[3])<2:
            raise ValueError
        self.number = ph
    
    @property
    def area_code(self):
        return self.number[:3]
    
    def pretty(self):
        ph = self.number
        return f'({ph[:3]}) {ph[3:6]}-{ph[6:]}'

ph = Phone('2234567890')
ph, ph.area_code, ph.pretty()

(<__main__.Phone at 0x10c27f2e8>, '223', '(223) 456-7890')

In [1]:
# wordy

In [1]:
def split_on_digit(q):
    f = 'X'
    fa = ''
    while f.isalpha():
        fa = fa + ' ' + f
        f, q = q.split(maxsplit=1)
    return fa[3:], int(f), q

def calculate(question):
    q = question.replace('?', ' ?')
    _, operand, rest = split_on_digit(q)
    
    is_continuing = True
    while is_continuing:
        operator, operand_2, rest = split_on_digit(rest)
        ops_dict = {'plus': operand + operand_2,
                   'minus': operand - operand_2,
                   'multiplied by': operand * operand_2,
                   'divided by': operand / operand_2}
        operand = ops_dict.get(operator, 'ERROR')
        if operand == 'ERROR':
            raise ValueError
        if rest == '?':
            is_continuing = False
    
    return operand

In [2]:
question = "What is -333 divided by 3 plus 1 minus -33?"
calculate(question)

-77.0

In [ ]:
# queen attack

In [1]:
def board(white_position, black_position):
    board = [['_']*8]*8
    for (r, c), ch in zip((white_position, black_position), ('W', 'B')):
        if min(r, c) < 0 or max(r, c) > 7 or white_position==black_position:
            raise ValueError
        rm = board[r][:]
        rm[c] = ch
        board[r] = rm
    return list(map(lambda r: ''.join(r), board))


def can_attack(white_position, black_position):
    wr, wc = white_position
    br, bc = black_position
    if min(wr, wc, br, bc) < 0 or max(wr, wc, br, bc) > 7 or (wr, wc)==(br, bc):
        raise ValueError
    return wr==br or wc==bc or abs(wr-br)==abs(wc-bc)

In [2]:
board((2, 3), (5, 6))

['________',
 '________',
 '___W____',
 '________',
 '________',
 '______B_',
 '________',
 '________']

In [ ]:
# minesweeper

In [1]:
def star_ct(r, c, inp):
    row_ct = len(inp)
    col_ct = len(inp[0])
    stars = 0
    for ri in [r-1, r, r+1]:
        for ci in [c-1, c, c+1]:
            if 0 <= ri < row_ct and 0 <= ci < col_ct:
                if inp[ri][ci] == '*':
                    stars += 1
    return str(stars) if stars else ' '

def board(input_board_array):
    if len(set(map(len, input_board_array))) > 1:
        raise ValueError('rows are not of equal length')
    out = []
    for r, row in enumerate(input_board_array):
        out_row = []
        for c, val in enumerate(row):
            if val == ' ':
                out_row.append(star_ct(r, c, input_board_array))
            elif val == '*':
                out_row.append('*')
            else:
                raise ValueError('invalid char encountered')
        out.append(''.join(out_row))
    return out

In [2]:
inp = ["***", "* *", "***"]
board(inp)

['***', '*8*', '***']

In [3]:
inp = [" * * "]
board(inp)

['1*2*1']

In [4]:
inp = [" ",
       "*",
       " ",
       "*",
       " "]
board(inp)

['1', '*', '2', '*', '1']

In [5]:
inp = ["*",
       " ",
       "  ",
       " ",
       "*"]
board(inp)

ValueError: rows are not of equal length

In [ ]:
# check_brackets

In [1]:
def check_brackets(input_string):
    br_opening = '{[('
    br_closing = '}])'
    br_closes = dict(zip(br_closing, br_opening))
    br_stack = []
    for ch in input_string:
        if ch in br_opening:
            br_stack.append(ch)
        if ch in br_closing:
            if br_stack and br_closes[ch] == br_stack[-1]:
                br_stack.pop()
            else:
                return False
    return br_stack == []
        


check_brackets("[({]})"), check_brackets("([{}({}[])])")

(False, True)